In [1]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl");

estimateΩEmpirically (generic function with 1 method)

In [5]:
n = 200
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [6, 4, 8, 12, 3, 8, 2, 7, 6, 4  …  8, 11, 3, 6, 9, 3, 9, 3, 3, 10]


In [3]:
## Run Louvains, Naive, and Faster version

@time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

# This may not be exactly the same, but that's okay.
# Even a difference of 1e-14 in modularity change may lead you to make a 
# different greedy decision, but it's still a valid move.
@show mlou, mlou2
@show abs(mlou- mlou2)


Louvain Iteration 1
Louvain Iteration 2
 18.083869 seconds (144.25 M allocations: 10.832 GiB, 10.27% gc time)

Louvain Iteration 1
Louvain Iteration 2
  1.400914 seconds (5.27 M allocations: 312.008 MiB, 5.65% gc time)
(mlou, mlou2) = (-37780.4478438872007249699103040123382424297915349598042666912078857421875, -37780.4478438872007249699103040123382424297915349598042666912078857421875)
abs(mlou - mlou2) = 0.0


0.0

In [4]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
Z = collect(1:n)
@time Znew, changed = SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
  2.341902 seconds (6.66 M allocations: 440.950 MiB, 4.93% gc time)


-37780.4478438872007249699103040123382424297915349598042666912078857421875

In [10]:
## Now repeatedly call the SuperNodeStep until no more improvement is possible
@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

SuperNode Louvain: Phase 1

Louvain Iteration 1
Louvain Iteration 2
SuperNode Louvain: Phase 2

Louvain Iteration 1
  0.543997 seconds (4.43 M allocations: 313.029 MiB, 12.48% gc time)


-36610.57871622490183330896128081899387751718677463941276073455810546875